In [1]:
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine, text
from pandas.tseries.offsets import BDay

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

In [3]:
today = date.today()
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-10-01
yesterday: 2025-09-30


In [5]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\Downloads\Datasets
dts_path = os.path.join(user_path, "Downloads", "Datasets")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")
#C:\Users\PC1\OneDrive\A5\Excel
xsl_path = os.path.join(base_path, "Excel")

In [7]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Excel path (xsl_path): {xsl_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Excel path (xsl_path): C:\Users\PC1\OneDrive\A5\Excel
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [9]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}', 
    
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    }

In [11]:
def show_buy(const, name):
    sql = """
    SELECT * 
    FROM buy 
    WHERE name = '{}'
    """.format(name)
    
    df_buy = pd.read_sql(sql, const)
    df_buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
    df_buy.rename(columns={'volbuy':'shares'},inplace=True)
    df_buy['shares'] = df_buy['shares'].astype('int64')
    return df_buy.style.format(format_dict)

In [13]:
def show_dividend(const, name):
    sql = """
    SELECT * 
    FROM dividend 
    WHERE name = '{}'
    """.format(name)
    
    df_dividend = pd.read_sql(sql, const)
    df_dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
    df_dividend.columns = df_dividend.columns.str.lower()
    df_dividend['shares'] = df_dividend['shares'].astype('int64')
    df_dividend['xdate'] = pd.to_datetime(df_dividend['xdate'])
    df_dividend['paiddate'] = pd.to_datetime(df_dividend['paiddate'])
    return df_dividend.style.format(format_dict)

In [15]:
def show_stocks(conlite, name):
    sql = """
    SELECT * 
    FROM stocks 
    WHERE name = '{}'
    """.format(name)
    
    df_stocks = pd.read_sql(sql, conlite)
    
    # Convert bytes columns to string with error handling
    for col in df_stocks.columns:
        if df_stocks[col].dtype == object and len(df_stocks) > 0:
            # Check if first value is bytes
            first_val = df_stocks[col].iloc[0] if len(df_stocks) > 0 else None
            if isinstance(first_val, bytes):
                df_stocks[col] = df_stocks[col].apply(
                    lambda x: x.decode('utf-8', errors='replace') if isinstance(x, bytes) else x
                )
    
    # Ensure numeric/date columns are correctly typed
    if 'date' in df_stocks.columns:
        df_stocks['date'] = pd.to_datetime(df_stocks['date'], errors='coerce')
    
    # Create a filtered format_dict based on column types
    filtered_format_dict = {}
    for col, fmt in format_dict.items():
        if col in df_stocks.columns:
            # Check if column is numeric for numeric formatting
            if ('{' in fmt and ('}' in fmt and (',' in fmt or '.' in fmt or '%' in fmt))):
                if pd.api.types.is_numeric_dtype(df_stocks[col]):
                    filtered_format_dict[col] = fmt
            # Check if column is datetime for date formatting
            elif '{:%' in fmt:
                if pd.api.types.is_datetime64_any_dtype(df_stocks[col]):
                    filtered_format_dict[col] = fmt
            # For simple string formatting without special characters
            else:
                filtered_format_dict[col] = fmt
    
    return df_stocks.style.format(filtered_format_dict)

In [17]:
def update_buy(const, name, new_qty, new_price):
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE buy
        SET volbuy = :new_qty,
        price = :new_price
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    result = const.execute(sqlUpd, {
        'new_qty': new_qty,
        'new_price': new_price,
        'name': name
    })

    return f"Records updated = {result.rowcount}"

In [19]:
def update_dividend(const, name, new_qty):
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE dividend
        SET shares = :new_qty
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    result = const.execute(sqlUpd, {
        'new_qty': new_qty,  # Use the scalar value
        'name': name
    })

    return f"Records updated = {result.rowcount}"

In [21]:
def update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty):  
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE stocks
        SET available_qty = :new_qty, 
            cost = :new_unit_cost,
            buy_target = :new_buy_target,
            sell_target = :new_sell_target,    
            qty = :new_buy_qty
        WHERE name = :name
    """)    
    # Execute the query with parameters
    result = conlite.execute(sqlUpd, {
        'new_qty': new_qty,  # Use the scalar value
        'new_unit_cost': new_unit_cost, # Use the scalar value 
        'new_buy_target': new_buy_target,
        'new_sell_target': new_sell_target,     
        'new_buy_qty': new_buy_qty,
        'name': name
    })
    return f"Records updated = {result.rowcount}"

### End of Definition

## Begin of  Buy transaction  

In [25]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
df_buys_latest = pd.read_sql(sql, conpf)
df_buys_latest.style.format(format_dict)

,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,ORI,1350,125,2025-09-08,"10,000",2.62,54.23,3.80,Sold,"26,258.03",SHORT,None


In [27]:
transaction = 'B'

names = df_buys_latest['name']
name = names.to_string(index=False)

sr_qty = df_buys_latest['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = df_buys_latest['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

print(transaction, name, qty, buy_price)

B ORI 10000 2.62


### Begin of Update buy table in MySQL database from buy transaction

In [30]:
sql = """
    SELECT *
    FROM buy 
    WHERE name = '{}'
""".format(name)
    
df_buy = pd.read_sql(sql, const)
df_buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
df_buy.rename(columns={'volbuy':'shares'},inplace=True)
df_buy['shares'] = df_buy['shares'].astype('int64')
df_buy.style.format(format_dict)

,name,date,shares,price,active,period,grade
0,ORI,2022-04-07,"50,000",9.00,1,4,C2


### In case there is the same stock name in port

In [33]:
show_buy(const, name)

,name,date,shares,price,active,period,grade
0,ORI,2022-04-07,"50,000",9.00,1,4,C2


In [35]:
new_qty = int(df_buy['shares'].iloc[0] + qty)
new_unit_cost = df_buy['price'].iloc[0] 
print(transaction, name, new_qty, new_unit_cost)

B ORI 60000 9.0


In [37]:
update_buy(const, name, new_qty, new_unit_cost)

'Records updated = 1'

In [39]:
show_buy(const, name)

,name,date,shares,price,active,period,grade
0,ORI,2022-04-07,"60,000",9.00,1,4,C2


### In case of totally new stock

In [24]:
if df_buy.empty:
    rcd = {
        'name': 'CPF',
        'date': '2025-08-21',
        'volbuy': 4000,
        'price': 24.1,
        'volsell': 0,
        'volbal': 0,
        'active': 1,
        'dividend': 0,
        'period': 4,
        'grade': 'A1'
    }
    sqlIns = text("""
    INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
    VALUES (:name, :date, :volbuy, :price, :volsell, :volbal, :active, :dividend, :period, :grade)
    """)
    result = const.execute(sqlIns, rcd)
    result.rowcount

In [25]:
show_buy(const, name)

,name,date,shares,price,active,period,grade
0,ORI,2022-04-07,"55,000",9.00,1,1,C2


### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [41]:
sql = """
    SELECT * 
    FROM dividend 
    WHERE name = '{}'
""".format(name)
    
df_dividend = pd.read_sql(sql, const)
df_dividend.columns = df_dividend.columns.str.lower()
df_dividend['shares'] = df_dividend['shares'].astype('int64')

df_dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,price,percent,shares,xdate,paiddate,kind,actual
0,ORI,0.0000,0.0210,0.0000,0.0000,0.0210,0.00,0.00%,"50,000",2025-03-13,2025-05-23,,0


In [43]:
show_dividend(const, name)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,ORI,0.0000,0.0210,0.0000,0.0000,0.0210,"50,000",2025-03-13,2025-05-23,,0


In [45]:
new_qty = df_dividend['shares'].iloc[0] + qty
print(name, new_qty)

ORI 60000


In [47]:
update_dividend(const, name, new_qty)

'Records updated = 1'

In [49]:
show_dividend(const, name)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,ORI,0.0000,0.0210,0.0000,0.0000,0.0210,"60,000",2025-03-13,2025-05-23,,0


### In case of totally new dividend

In [34]:
if df_dividend.empty:
    rcd = {
        'name': 'CPF',
        'q4': '0',
        'q3': '0',
        'q2': '1.00',
        'q1': '0',    
        'dividend': '1.00',      
        'shares': 4000,
        'xdate': '2025-08-29',
        'paiddate': '2025-09-12',
        'kind': ' ',
        'actual': 1
    }
    sqlIns = text("""
    INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
    VALUES (:name, :q4, :q3, :q2, :q1, :dividend, :shares, :xdate, :paiddate, :kind, :actual)
    """)
    print(sqlIns)

In [35]:
if df_dividend.empty:
    result = const.execute(sqlIns, rcd)
    result.rowcount

In [36]:
show_dividend(const, name)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,ORI,0.0000,0.0210,0.0000,0.0000,0.0210,"55,000",2025-03-13,2025-05-23,,0


### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [59]:
sql = """
    SELECT name, cost, available_qty
    FROM stocks
    WHERE name = '{}'
""".format(name)

df_stocks = pd.read_sql(sql, conlite)
df_stocks['available_qty'] = df_stocks['available_qty'].astype('int64')
df_stocks

,name,cost,available_qty
0,ORI,9.0,50000


In [61]:
show_stocks(conlite, name)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,116,ORI,0.00,0.00,B,2.00,9.00,0.00,0.00,9.00,"10,000",-4,4,"50,000",0.000000,0.000000,RD70Pct,SET100


In [63]:
new_qty = int(df_stocks['available_qty'].iloc[0] + qty)
new_qty

60000

In [65]:
new_unit_cost = df_stocks['cost'].iloc[0]
new_unit_cost

9.0

In [67]:
new_buy_target = 2.00
new_sell_target = 9.00
new_buy_qty = 10_000
print(transaction, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty)

B ORI 60000 9.0 2.0 9.0 10000


In [69]:
update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty)

'Records updated = 1'

In [71]:
show_stocks(conlite, name)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,116,ORI,0.00,0.00,B,2.00,9.00,0.00,0.00,9.00,"10,000",-4,4,"60,000",0.000000,0.000000,RD70Pct,SET100


### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy transaction

## Begin of Sale transaction

In [73]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
df_sells_latest = pd.read_sql(sql, conpf)
df_sells_latest.style.format(format_dict)

,name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,ORI,125,2025-09-08,"10,000",2.62,1117,1350,2025-10-01,2.72,56.30,3.94,"27,139.76",23,881.73,3.36%,53.29%,-117,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [76]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = df_sells_latest['name']
name = names.to_string(index=False)

sr_qty = df_sells_latest['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = df_sells_latest['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = df_sells_latest['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = df_sells_latest['buy_id']
buy_id = buy_ids.to_string(index=False)
print(name, qty, buy_price, sell_price)

ORI -10000 2.62 2.72


### Begin of Update buy table in MySQL stock database from sale transaction

In [79]:
sql = """
    SELECT *
    FROM buy 
    WHERE name = '{}'
""".format(name)
    
df_buy = pd.read_sql(sql, const)
df_buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
df_buy.rename(columns={'volbuy':'shares'},inplace=True)
df_buy['shares'] = df_buy['shares'].astype('int64')
df_buy.style.format(format_dict)

,name,date,shares,price,active,period,grade
0,ORI,2022-04-07,"60,000",9.00,1,4,C2


In [81]:
old_qty = int(df_buy['shares'].iloc[0])
old_price = df_buy['price'].iloc[0]
old_qty, old_price 

(60000, 9.0)

In [83]:
old_cost_amt = old_qty * old_price
sales_amt = qty * sell_price
new_qty = old_qty + qty
new_cost_amt = old_cost_amt + sales_amt
#new_unit_cost = new_cost_amt / new_qty
new_unit_cost = old_price
print(name, old_cost_amt, sales_amt, new_cost_amt, new_qty, new_unit_cost)

ORI 540000.0 -27200.000000000004 512800.0 50000 9.0


In [85]:
show_buy(const, name)

,name,date,shares,price,active,period,grade
0,ORI,2022-04-07,"60,000",9.00,1,4,C2


In [87]:
print(transaction, name, new_qty, new_unit_cost)

S ORI 50000 9.0


In [89]:
update_buy(const, name, new_qty, new_unit_cost)

'Records updated = 1'

In [91]:
show_buy(const, name)

,name,date,shares,price,active,period,grade
0,ORI,2022-04-07,"50,000",9.00,1,4,C2


In [59]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM buy
    WHERE name = :name AND volbuy = 0 AND active = 0;
""")

# Execute the query with the correct parameter dictionary
result = const.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 0


### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [93]:
sql = """
    SELECT * 
    FROM dividend 
    WHERE name = '{}'
""".format(name)
    
df_dividend = pd.read_sql(sql, const)
df_dividend.columns = df_dividend.columns.str.lower()
df_dividend['shares'] = df_dividend['shares'].astype('int64')

df_dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,price,percent,shares,xdate,paiddate,kind,actual
0,ORI,0.0000,0.0210,0.0000,0.0000,0.0210,0.00,0.00%,"60,000",2025-03-13,2025-05-23,,0


In [95]:
new_qty = df_dividend['shares'].iloc[0] + qty
new_qty

50000

In [97]:
show_dividend(const, name)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,ORI,0.0000,0.0210,0.0000,0.0000,0.0210,"60,000",2025-03-13,2025-05-23,,0


In [99]:
update_dividend(const, name, new_qty)

'Records updated = 1'

In [101]:
show_dividend(const, name)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,ORI,0.0000,0.0210,0.0000,0.0000,0.0210,"50,000",2025-03-13,2025-05-23,,0


In [67]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM dividend
    WHERE name = :name AND shares = 0 AND actual = 0;
""")

# Execute the query with the correct parameter dictionary
result = const.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 0


In [68]:
show_dividend(const, name)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,ORI,0.0000,0.0210,0.0000,0.0000,0.0210,"50,000",2025-03-13,2025-05-23,,0


### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [103]:
sql = """
    SELECT name, cost, available_qty
    FROM stocks
    WHERE name = '{}'
""".format(name)

df_stocks = pd.read_sql(sql, conlite)
df_stocks['available_qty'] = df_stocks['available_qty'].astype('int64')
df_stocks

,name,cost,available_qty
0,ORI,9.0,60000


In [105]:
old_qty = int(df_stocks['available_qty'].iloc[0])
new_qty = old_qty + qty
new_buy_target = 2.00
new_sell_target = 9.00
new_buy_qty = 10_000

In [107]:
show_stocks(conlite, name)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,116,ORI,0.00,0.00,B,2.00,9.00,0.00,0.00,9.00,"10,000",-4,4,"60,000",0.000000,0.000000,RD70Pct,SET100


In [109]:
print(name, transaction, new_qty, buy_price, new_buy_target, new_sell_target, new_buy_qty)

ORI S 50000 2.62 2.0 9.0 10000


In [111]:
update_stock(conlite, name, new_qty, buy_price, new_buy_target, new_sell_target, new_buy_qty)

'Records updated = 1'

In [113]:
show_stocks(conlite, name)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,116,ORI,0.00,0.00,B,2.00,9.00,0.00,0.00,2.62,"10,000",-4,4,"50,000",0.000000,0.000000,RD70Pct,SET100


### End of Update stocks table in SQLite port_lite database from sale transaction

## End of Sale process

### =====================================================

In [80]:
#price_date = '2025-02-14'
sql = "SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, \
FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,\
FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, \
FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, \
FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol \
FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '{}' \
ORDER BY period, buy.name".format(yesterday)
print(sql)

SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '2025-08-27' ORDER BY period, buy.name


In [81]:
df_buy = pd.read_sql(sql, const)
df_buy

,period,grade,name,date,volbuy,buy_price,mkt_price,amtbuy,amtmkt,amtpol,pctpol
0,1,C1,AH,2023-06-08,"1,200",37.00,14.50,"44,400.00","17,400.00","-27,000.00",-60.81
1,1,A1,AWC,2023-06-15,"9,000",4.96,2.34,"44,640.00","21,060.00","-23,580.00",-52.82
2,1,A2,JMART,2023-01-20,"6,800",33.00,8.35,"224,400.00","56,780.00","-167,620.00",-74.70
3,1,C2,ORI,2022-04-07,"50,000",9.00,2.20,"450,000.00","110,000.00","-340,000.00",-75.56
4,1,C2,PTTGC,2021-03-17,"6,000",64.75,26.50,"388,500.00","159,000.00","-229,500.00",-59.07
5,1,C2,SCC,2021-09-20,600,405.00,215.00,"243,000.00","129,000.00","-114,000.00",-46.91
6,1,A3,SINGER,2023-01-19,"6,000",24.80,5.50,"148,800.00","33,000.00","-115,800.00",-77.82
7,2,A1,AIMIRT,2023-08-17,"17,500",10.60,9.70,"185,500.00","169,750.00","-15,750.00",-8.49
8,2,C1,CPNREIT,2022-08-16,"55,000",18.00,11.20,"990,000.00","616,000.00","-374,000.00",-37.78
9,2,C1,DIF,2020-08-01,"50,000",12.17,8.20,"608,500.00","410,000.00","-198,500.00",-32.62


In [82]:
file_name = 'portfolio.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [83]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\portfolio.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\portfolio.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\portfolio.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\portfolio.csv


In [84]:
df_buy.to_csv(output_file, header=True, index=False)
df_buy.to_csv(icd_file, header=True, index=False)
df_buy.to_csv(god_file, header=True, index=False)
df_buy.to_csv(osd_file, header=True, index=False)

In [85]:
file_name = '035-portfolio.xlsx'
xsl_file = os.path.join(xsl_path, file_name)
df_buy.to_excel(xsl_file, index=False)

In [152]:
conlite.commit()